<a href="https://colab.research.google.com/github/andrsbtrg/S9_ML/blob/main/s9_tensorflowtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount your Google drive to access files:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Install dependencies

In [ ]:
!pip install tensorflow-gpu==2.0.0

Run to check tensorflow works

In [ ]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

Importing libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import json
from sklearn.model_selection import train_test_split
from google.colab.patches import cv2_imshow

Function to load images

In [ ]:
def load_images(img_paths, target_size, interpolation='nearest', color_mode='rgb', normalize=False):
    images = []
    for img_path in img_paths:
        #load image
        img = keras.preprocessing.image.load_img(img_path, color_mode=color_mode, target_size=target_size, interpolation=interpolation)
        
        #convert to array
        img = keras.preprocessing.image.img_to_array(img)
        
        #normalize
        if normalize:
            img = img / 255
        
        #append
        images.append(img)
    
    images = np.stack(images, axis=0)
    
    return images

Loading the data:

Make sure that the path point to the specific folder "01_shapes_shuffled" inside the folder "dataset" in your drive



In [ ]:
#=======================
#LOAD DATA
#=======================
#PATHS

#current path
current_folder = Path.cwd()

#dataset path
dataset_path = current_folder.joinpath("/content/drive/MyDrive/s9_dataset/01_shapes_shuffled")
print(dataset_path)

Loading the images and the labels:

In [ ]:
#images path
img_paths = dataset_path.glob("*.png")

img_paths_list = []
for path in img_paths:
    path_str = str(path)
    img_paths_list.append(path_str)

# print(len(img_paths_list))
# print(img_paths_list[:10])

#list02 = [str(path) for path in img_paths]

#Load labels
labels_file = open("/content/drive/MyDrive/s9_dataset/labels_01_shapes_shuffled.json")
labels = json.load(labels_file)

labels = np.stack(labels, axis =0)
#---------------------
#LOAD IMAGES

# #load image
img = keras.preprocessing.image.load_img(img_paths_list[9], color_mode='grayscale', target_size=(28, 28), interpolation='bicubic') #normally we use "nearest", 
# print(type(img))
# print(img.size)

# #convert to array
img = keras.preprocessing.image.img_to_array(img)
# print(type(img))
#print(img.shape)

# #normalize
#img = img / 255

# #preview image
#plt.imshow(img)
images = load_images(img_paths_list, target_size=(28,28), interpolation='bicubic', color_mode='grayscale', normalize=True)

#-----------------------
#SPLIT DATA
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=40)
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

#=======================
#ARTIFICIAL NEURAL NETWORK
#=======================



---
Architecture of the model


In [ ]:
ann_model = keras.Sequential([
                              # Input Layer
                              keras.layers.Flatten(input_shape = (28,28,1)),
                              # Hidden Layer
                              keras.layers.Dense(units=100, activation = 'relu'),
                              # Hidden Layer
                              keras.layers.Dense(units = 5, activation= 'softmax'),
])

Set the Optimizer

In [ ]:
ann_model.compile(optimizer = 'Adam',
                  loss = 'sparse_categorical_crossentropy',
                  metrics = ['accuracy'],
                  )

In [ ]:
# Summary

ann_model.summary()

Training

In [ ]:
model_history = ann_model.fit(x_train,y_train,epochs=100)

Visualize epoch vs accuracy of the model

In [ ]:
plt.figure()
plt.plot(model_history.history['accuracy'],label='accuracy')
plt.xlabel("epoch")
plt.ylabel("accuracy")
plt.ylim([0,1])

---

Evaluate 



In [ ]:
ann_model.evaluate(x_test,y_test)

Saving model

In [ ]:
ann_model.save('/content/drive/MyDrive/s9_dataset/saved_model/ann_model_01')

Loading model

In [ ]:
model_saved = keras.models.load_model('/content/drive/MyDrive/s9_dataset/saved_model/ann_model_01')
model_saved.summary()

In [ ]:
model_saved.evaluate(x_test,y_test)

In [ ]:
img_zero = np.zeros(x_test[5].shape)
plt.figure()
plt.imshow(tf.squeeze(img_zero)) #using tf.squeeze on the image fixes the plt error

In [ ]:
def predict_image(x,y,img_index,model,label_list):
  # predict single image
  img = x[img_index]

  y_predict = model_saved.predict(x_test[5:6])
  y_predict_class = np.argmax(y_pred)
  y_predict_class = label_list[y_predict_class]

  # real label
  y_real = y[img_index]
  y_real = label_list[y_real]

  # plot
  plt.figure(figsize=(5,5))
  plt.imshow(tf.squeeze(img)) #squeeze the image before displaying
  plt.xlabel("pred: {}, label: {}".format(y_predict_class,y_real))


predict_image(x_test,y_test,1,model_saved,labels)